## This notebook aimes to try diffrernt summarization models from hugging face, We will try: <br>
- PEGASUS (English summary) <br>
- BAART (English summary) <br>
- T5 (Multilangual summary) <br>

In [1]:
PEGASUS = "google/pegasus-cnn_dailymail"
BART = "facebook/bart-large-cnn"
T5 = "csebuetnlp/mT5_multilingual_XLSum"

In [2]:
English_text ='''
Remote work expanded quickly after many companies invested in cloud tools and online collaboration.
Employees often report benefits such as reduced commuting time, more flexible schedules, and improved
focus for individual tasks. However, managers also highlight challenges, including weaker informal
communication, difficulty onboarding new hires, and increased security risks if personal devices are 
not properly managed. As a result, many organizations are adopting hybrid policies, combining in-office
days for teamwork with remote days for deep work, while updating guidelines for security, meetings, 
and performance evaluation
'''

In [3]:
Arabic_text = '''
شهدت المدن الكبرى في السنوات الأخيرة توسعًا سريعًا في استخدام تطبيقات 
توصيل الطعام، وهو ما غيّر عادات الشراء لدى كثير من الأسر. من جهة، وفّرت هذه الخدمات وقتًا وجهدًا وساعدت مطاعم صغيرة على الوصول لعملاء جدد. 
ومن جهة أخرى، ارتفعت شكاوى بعض المستخدمين من زيادة الرسوم وتفاوت جودة الخدمة وتأخر الطلبات في أوقات الذروة. كما أشار مختصون إلى أن الاعتماد 
الزائد على الوجبات الجاهزة قد يؤثر على العادات الغذائية، ما يدفع بعض المبادرات المحلية لتشجيع خيارات صحية وتقديم معلومات أوضح عن المكونات والسعرات.
'''

## **PEGASUS**

In [4]:
!pip install transformers==4.52.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 86.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.5 MB/s eta 0:00:00:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1


In [23]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import re

In [8]:
tokenizer_p = AutoTokenizer.from_pretrained(PEGASUS)
model_p = AutoModelForSeq2SeqLM.from_pretrained(PEGASUS)

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

2026-01-08 11:45:33.974098: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767872734.196591      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767872734.260014      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767872734.784820      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767872734.784856      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767872734.784860      55 computation_placer.cc:177] computation placer alr

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [14]:
input_English = tokenizer_p(
    English_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=200
)

summary_ids_E = model_p.generate(
    **input_English,
    max_new_tokens=200,
    num_beams=4,
    no_repeat_ngram_size=3
)

print(tokenizer_p.decode(summary_ids_E[0], skip_special_tokens=True))

Employees report benefits such as reduced commuting time, more flexible schedules, and improved focus on individual tasks .<n>However, managers also highlight challenges, including weaker informal communication, difficulty onboard new hires, and increased security risks .


In [15]:
input_Arabic = tokenizer_p(
    Arabic_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=200
)

summary_ids_A = model_p.generate(
    **input_Arabic,
    max_new_tokens=200,
    num_beams=4,
    no_repeat_ngram_size=3
)

print(tokenizer_p.decode(summary_ids_A[0], skip_special_tokens=True))

 "  "<n> "' '"<n>' " "''' .<n>'' "'" .


### **As shown, this model is effecient on English text but not suitable for Arabic text**

## **BART**

In [16]:
tokenizer_b = AutoTokenizer.from_pretrained(BART)
model_b = AutoModelForSeq2SeqLM.from_pretrained(BART)

config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [17]:
input_English = tokenizer_b(
    English_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=200
)

summary_ids_E = model_b.generate(
    **input_English,
    max_new_tokens=200,
    num_beams=4,
    no_repeat_ngram_size=3
)

print(tokenizer_b.decode(summary_ids_E[0], skip_special_tokens=True))

Employees often report benefits such as reduced commuting time, more flexible schedules, and improved focus for individual tasks. However, managers also highlight challenges, including weaker informalcommunication and difficulty onboarding new hires. As a result, many organizations are adopting hybrid policies, combining in-office days for teamwork with remote days for deep work.


In [18]:
input_Arabic = tokenizer_b(
    Arabic_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=200
)

summary_ids_A = model_b.generate(
    **input_Arabic,
    max_new_tokens=200,
    num_beams=4,
    no_repeat_ngram_size=3
)

print(tokenizer_b.decode(summary_ids_A[0], skip_special_tokens=True))

                شهدت    ‘’’ ‘”’  “”  ”” ”,’,” ”,  , ”. “, ”.


### **As shown also, this model is effecient on English text but not suitable for Arabic text**

## **T5**

In [19]:
tokenizer_t = AutoTokenizer.from_pretrained(T5)
model_t = AutoModelForSeq2SeqLM.from_pretrained(T5)

tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/730 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.33G [00:00<?, ?B/s]

In [21]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_55/391913681.py:1: SyntaxWarning: invalid escape sequence '\s'
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))


In [25]:
input_ids = tokenizer_t(
    [WHITESPACE_HANDLER(Arabic_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model_t.generate(
    input_ids=input_ids,
    max_length=200,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer_t.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

حذّرت وزارة الصحة البريطانية من أن استخدام تطبيقات توصيل الطعام قد يؤثر على العادات الغذائية.


In [26]:
input_ids = tokenizer_t(
    [WHITESPACE_HANDLER(English_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = model_t.generate(
    input_ids=input_ids,
    max_length=200,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

summary = tokenizer_t.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

Remote work is increasingly being used as a way of improving employment, according to new research.


### **This model works on both English and Arabic Text**